<a href="https://colab.research.google.com/github/berthine/masakhane-mt/blob/master/benchmarks/en-kwy/jw300-baseline/berthine_en_kwy_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Masakhane - Machine Translation for African Languages (Using JoeyNMT)

### Languages: English-Kikongo

### Author: Berthine Nyunga Mpinda

## Retrieve your data & make a parallel corpus

If you are wanting to use the JW300 data referenced on the Masakhane website or in our GitHub repo, you can use `opus-tools` to convert the data into a convenient format. `opus_read` from that package provides a convenient tool for reading the native aligned XML files and to convert them to TMX format. The tool can also be used to fetch relevant files from OPUS on the fly and to filter the data as necessary. [Read the documentation](https://pypi.org/project/opustools-pkg/) for more details.

Once you have your corpus files in TMX format (an xml structure which will include the sentences in your target language and your source language in a single file), we recommend reading them into a pandas dataframe. Thankfully, Jade wrote a silly `tmx2dataframe` package which converts your tmx file to a pandas dataframe. 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
# TODO: Set your source and target languages. Keep in mind, these traditionally use language codes as found here:
# These will also become the suffix's of all vocab and corpus files used throughout
import os
source_language = "en"
target_language = "kwy" 
lc = False  # If True, lowercase the data.
seed = 42  # Random seed for shuffling.
tag = "baseline" # Give a unique name to your folder - this is to ensure you don't rewrite any models you've already submitted

os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language
os.environ["tag"] = tag

# This will save it to a folder in our gdrive instead!
!mkdir -p "/content/drive/My Drive/masakhane/$src-$tgt-$tag"
os.environ["gdrive_path"] = "/content/drive/My Drive/masakhane/%s-%s-%s" % (source_language, target_language, tag)

In [3]:
!echo $gdrive_path

/content/drive/My Drive/masakhane/en-kwy-baseline


In [4]:
# Install opus-tools
! pip install opustools-pkg

     |████████████████████████████████| 81kB 2.2MB/s 


In [5]:
# Downloading our corpus
! opus_read -d JW300 -s $src -t $tgt -wm moses -w jw300.$src jw300.$tgt -q

# extract the corpus file
! gunzip JW300_latest_xml_$src-$tgt.xml.gz


Alignment file /proj/nlpl/data/OPUS/JW300/latest/xml/en-kwy.xml.gz not found. The following files are available for downloading:

   2 MB https://object.pouta.csc.fi/OPUS-JW300/v1/xml/en-kwy.xml.gz
 263 MB https://object.pouta.csc.fi/OPUS-JW300/v1/xml/en.zip
  17 MB https://object.pouta.csc.fi/OPUS-JW300/v1/xml/kwy.zip

 282 MB Total size
./JW300_latest_xml_en-kwy.xml.gz ... 100% of 2 MB
./JW300_latest_xml_en.zip ... 100% of 263 MB
./JW300_latest_xml_kwy.zip ... 100% of 17 MB


In [6]:
# Download the global test set.
! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-any.en
  
# And the specific test set for this language pair.
os.environ["trg"] = target_language 
os.environ["src"] = source_language 

! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-$trg.en 
! mv test.en-$trg.en test.en
! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-$trg.$trg 
! mv test.en-$trg.$trg test.$trg

--2020-08-23 17:35:31--  https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-any.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 277791 (271K) [text/plain]
Saving to: ‘test.en-any.en’

test.en-any.en      100%[===================>] 271.28K  --.-KB/s    in 0.05s   

2020-08-23 17:35:31 (5.18 MB/s) - ‘test.en-any.en’ saved [277791/277791]

--2020-08-23 17:35:32--  https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-kwy.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Len

In [7]:
# Read the test data to filter from train and dev splits.
# Store english portion in set for quick filtering checks.
en_test_sents = set()
filter_test_sents = "test.en-any.en"
j = 0
with open(filter_test_sents) as f:
  for line in f:
    en_test_sents.add(line.strip())
    j += 1
print('Loaded {} global test sentences to filter from the training/dev data.'.format(j))

Loaded 3571 global test sentences to filter from the training/dev data.


In [9]:
!ls

drive	   JW300_latest_xml_en-kwy.xml	sample_data	test.kwy
jw300.en   JW300_latest_xml_en.zip	test.en
jw300.kwy  JW300_latest_xml_kwy.zip	test.en-any.en


In [10]:
import pandas as pd

# TMX file to dataframe
source_file = 'jw300.' + source_language
target_file = 'jw300.' + target_language

source = []
target = []
skip_lines = []  # Collect the line numbers of the source portion to skip the same lines for the target portion.
with open(source_file) as f:
    for i, line in enumerate(f):
        # Skip sentences that are contained in the test set.
        if line.strip() not in en_test_sents:
            source.append(line.strip())
        else:
            skip_lines.append(i)             
with open(target_file) as f:
    for j, line in enumerate(f):
        # Only add to corpus if corresponding source was not skipped.
        if j not in skip_lines:
            target.append(line.strip())
    
print('Loaded data and skipped {}/{} lines since contained in test set.'.format(len(skip_lines), i))
    
df = pd.DataFrame(zip(source, target), columns=['source_sentence', 'target_sentence'])
# if you get TypeError: data argument can't be an iterator is because of your zip version run this below
#df = pd.DataFrame(list(zip(source, target)), columns=['source_sentence', 'target_sentence'])
df.head(10)

Loaded data and skipped 5241/193639 lines since contained in test set.


,source_sentence,target_sentence
0,Jehovah’s Word Is Alive,Diambu dia Yave Diamoyo
1,Highlights From the Book of Joshua,Nsasa za Nkand’a Yosua
2,ENCAMPED on the Plains of Moab in 1473 B.C.E ....,VAVA Aneyisaele balwaka muna Ndimb’a Moabe mun...
3,Their 40 - year wilderness sojourn is about to...,( Yosua 1 : ​ 11 ) E nkangalu wa ya 40 ma mvu ...
4,"A little over two decades later , the leader J...","Vioka makumole ma mvu , o Yosua wa mfidi a Isa..."
5,And Jehovah your God was the one who kept push...,"O Yave wa Nzambi eno , yandi okubakûla vovo nw..."
6,"Written by Joshua in 1450 B.C.E . , the book o...",Muna Nkand’a Yosua muna ye tusansu twamfunu tw...
7,As we stand at the threshold of the promised n...,Ekolo tufinamang’o kota muna nz’ampa eyi yasil...
8,"With keen interest , then , let us give attent...",Yambula twabadika o nkand’a Yosua ye sungididi...
9,TO “ THE DESERT PLAINS OF JERICHO ”,MUNA “ NDIMB’A YERIKO ”


In [11]:
df.source_sentence[100]

'Later , Joshua assembles all the tribes of Israel at Shechem .'

In [12]:
df.target_sentence[100]

'Kuna kwalanda , Yosua walunganesa makanda mawonso ma Isaele kuna Sekeme .'

In [13]:
len(df)

188399

## Pre-processing and export

It is generally a good idea to remove duplicate translations and conflicting translations from the corpus. In practice, these public corpora include some number of these that need to be cleaned.

In addition we will split our data into dev/test/train and export to the filesystem.

In [14]:
# drop duplicate translations
df_pp = df.drop_duplicates()

# drop conflicting translations
# (this is optional and something that you might want to comment out 
# depending on the size of your corpus)
df_pp.drop_duplicates(subset='source_sentence', inplace=True)
df_pp.drop_duplicates(subset='target_sentence', inplace=True)

# Shuffle the data to remove bias in dev set selection.
df_pp = df_pp.sample(frac=1, random_state=seed).reset_index(drop=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [15]:
# Install fuzzy wuzzy to remove "almost duplicate" sentences in the
# test and training sets.
! pip install fuzzywuzzy
! pip install python-Levenshtein
import time
from fuzzywuzzy import process
import numpy as np

# reset the index of the training set after previous filtering
df_pp.reset_index(drop=False, inplace=True)

# Remove samples from the training data set if they "almost overlap" with the
# samples in the test set.

# Filtering function. Adjust pad to narrow down the candidate matches to
# within a certain length of characters of the given sample.
def fuzzfilter(sample, candidates, pad):
  candidates = [x for x in candidates if len(x) <= len(sample)+pad and len(x) >= len(sample)-pad] 
  if len(candidates) > 0:
    return process.extractOne(sample, candidates)[1]
  else:
    return np.nan

# NOTE - This might run slow depending on the size of your training set. We are
# printing some information to help you track how long it would take. 
scores = []
start_time = time.time()
for idx, row in df_pp.iterrows():
  scores.append(fuzzfilter(row['source_sentence'], list(en_test_sents), 5))
  if idx % 1000 == 0:
    hours, rem = divmod(time.time() - start_time, 3600)
    minutes, seconds = divmod(rem, 60)
    print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds), "%0.2f percent complete" % (100.0*float(idx)/float(len(df_pp))))

# Filter out "almost overlapping samples"
df_pp['scores'] = scores
df_pp = df_pp[df_pp['scores'] < 95]

     |████████████████████████████████| 51kB 1.7MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.0-cp36-cp36m-linux_x86_64.whl size=144796 sha256=660c97b30f99599e162882e9329a5acbbc133875c032467abfccdf8235973176
  Stored in directory: /root/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
Successfully built python-Levenshtein
00:00:00.08 0.00 percent complete
00:00:24.81 0.59 percent complete
00:00:49.43 1.17 percent complete
00:01:15.32 1.76 percent complete
00:01:40.78 2.34 percent complete
00:02:05.72 2.93 percent complete


00:02:30.47 3.51 percent complete
00:02:55.23 4.10 percent complete
00:03:19.60 4.68 percent complete
00:03:44.79 5.27 percent complete


00:04:09.32 5.85 percent complete
00:04:33.91 6.44 percent complete
00:04:58.29 7.02 percent complete
00:05:23.59 7.61 percent complete
00:05:48.19 8.20 percent complete
00:06:12.17 8.78 percent complete
00:06:36.90 9.37 percent complete
00:07:00.78 9.95 percent complete
00:07:25.35 10.54 percent complete
00:07:50.91 11.12 percent complete
00:08:15.51 11.71 percent complete
00:08:39.43 12.29 percent complete
00:09:04.19 12.88 percent complete
00:09:29.10 13.46 percent complete
00:09:53.19 14.05 percent complete
00:10:17.70 14.63 percent complete
00:10:41.99 15.22 percent complete
00:11:06.92 15.80 percent complete
00:11:31.73 16.39 percent complete
00:11:56.16 16.98 percent complete
00:12:20.76 17.56 percent complete
00:12:45.79 18.15 percent complete
00:13:11.18 18.73 percent complete
00:13:35.50 19.32 percent complete
00:13:59.53 19.90 percent complete
00:14:25.25 20.49 percent complete
00:14:49.75 21.07 percent complete
00:15:15.11 21.66 percent complete
00:15:39.30 22.24 percent co

00:28:00.45 39.80 percent complete
00:28:25.12 40.39 percent complete
00:28:49.95 40.98 percent complete
00:29:13.62 41.56 percent complete
00:29:37.46 42.15 percent complete
00:30:01.60 42.73 percent complete
00:30:26.17 43.32 percent complete
00:30:52.65 43.90 percent complete
00:31:17.02 44.49 percent complete
00:31:40.99 45.07 percent complete
00:32:05.46 45.66 percent complete
00:32:29.98 46.24 percent complete
00:32:54.56 46.83 percent complete
00:33:18.75 47.41 percent complete
00:33:42.41 48.00 percent complete
00:34:07.22 48.59 percent complete
00:34:31.20 49.17 percent complete
00:34:56.26 49.76 percent complete
00:35:20.78 50.34 percent complete
00:35:45.23 50.93 percent complete
00:36:09.72 51.51 percent complete
00:36:33.63 52.10 percent complete
00:36:57.67 52.68 percent complete
00:37:22.16 53.27 percent complete


00:37:46.71 53.85 percent complete
00:38:10.85 54.44 percent complete
00:38:34.51 55.02 percent complete
00:38:59.36 55.61 percent complete
00:39:23.85 56.20 percent complete
00:39:47.87 56.78 percent complete
00:40:13.02 57.37 percent complete
00:40:38.29 57.95 percent complete
00:41:02.88 58.54 percent complete


00:41:27.90 59.12 percent complete
00:41:52.33 59.71 percent complete
00:42:17.24 60.29 percent complete
00:42:41.63 60.88 percent complete
00:43:06.17 61.46 percent complete
00:43:31.04 62.05 percent complete
00:43:54.91 62.63 percent complete
00:44:20.43 63.22 percent complete
00:44:44.52 63.81 percent complete
00:45:09.14 64.39 percent complete
00:45:32.88 64.98 percent complete
00:45:56.96 65.56 percent complete
00:46:21.50 66.15 percent complete
00:46:45.81 66.73 percent complete
00:47:10.36 67.32 percent complete
00:47:36.27 67.90 percent complete
00:47:59.92 68.49 percent complete
00:48:24.85 69.07 percent complete
00:48:49.40 69.66 percent complete
00:49:13.56 70.24 percent complete
00:49:38.38 70.83 percent complete
00:50:03.28 71.41 percent complete
00:50:27.58 72.00 percent complete
00:50:52.89 72.59 percent complete
00:51:17.20 73.17 percent complete
00:51:41.96 73.76 percent complete
00:52:06.94 74.34 percent complete
00:52:30.92 74.93 percent complete
00:52:55.36 75.51 pe

00:56:35.86 80.78 percent complete
00:57:00.41 81.37 percent complete
00:57:26.19 81.95 percent complete
00:57:50.54 82.54 percent complete
00:58:15.44 83.12 percent complete
00:58:39.95 83.71 percent complete
00:59:04.72 84.29 percent complete
00:59:28.78 84.88 percent complete
00:59:53.34 85.46 percent complete
01:00:17.75 86.05 percent complete
01:00:42.75 86.63 percent complete
01:01:07.27 87.22 percent complete
01:01:31.51 87.81 percent complete
01:01:55.87 88.39 percent complete
01:02:20.11 88.98 percent complete
01:02:44.34 89.56 percent complete
01:03:08.83 90.15 percent complete
01:03:33.38 90.73 percent complete
01:03:58.60 91.32 percent complete


01:04:23.47 91.90 percent complete
01:04:48.53 92.49 percent complete
01:05:13.32 93.07 percent complete
01:05:37.43 93.66 percent complete
01:06:01.90 94.24 percent complete
01:06:25.31 94.83 percent complete
01:06:49.66 95.41 percent complete
01:07:15.45 96.00 percent complete
01:07:40.10 96.59 percent complete
01:08:04.11 97.17 percent complete
01:08:28.66 97.76 percent complete
01:08:52.23 98.34 percent complete
01:09:17.02 98.93 percent complete
01:09:41.18 99.51 percent complete


In [16]:
# This section does the split between train/dev for the parallel corpora then saves them as separate files
# We use 1000 dev test and the given test set.
import csv

# Do the split between dev/train and create parallel corpora
num_dev_patterns = 1000

# Optional: lower case the corpora - this will make it easier to generalize, but without proper casing.
if lc:  # Julia: making lowercasing optional
    df_pp["source_sentence"] = df_pp["source_sentence"].str.lower()
    df_pp["target_sentence"] = df_pp["target_sentence"].str.lower()

# Julia: test sets are already generated
dev = df_pp.tail(num_dev_patterns) # Herman: Error in original
stripped = df_pp.drop(df_pp.tail(num_dev_patterns).index)

with open("train."+source_language, "w") as src_file, open("train."+target_language, "w") as trg_file:
  for index, row in stripped.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")
    
with open("dev."+source_language, "w") as src_file, open("dev."+target_language, "w") as trg_file:
  for index, row in dev.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")

#stripped[["source_sentence"]].to_csv("train."+source_language, header=False, index=False)  # Herman: Added `header=False` everywhere
#stripped[["target_sentence"]].to_csv("train."+target_language, header=False, index=False)  # Julia: Problematic handling of quotation marks.

#dev[["source_sentence"]].to_csv("dev."+source_language, header=False, index=False)
#dev[["target_sentence"]].to_csv("dev."+target_language, header=False, index=False)

# Doublecheck the format below. There should be no extra quotation marks or weird characters.
! head train.*
! head dev.*

==> train.en <==
* However , the family study is not the only time to teach children .
However , reaching out for privileges in the congregation does mean striving to meet the qualifications set out in the Scriptures .
How did Adam use his free will in a good way ?
Should such situations cause you to conclude that it was a mistake to start on the journey and that you should abandon the automobile ?
From studying the Bible , I came to understand that I have a responsibility to the Giver of life , Jehovah .
Many people in Tuva live in remote communities that are hard to reach with the Kingdom message .
Why might the Israelites have been fearful at seeing the glory of God that Moses reflected ?
Although we ‘ hate every false path , ’ we may need to ask God to act in our behalf so that we do not succumb to some temptation to break his law .
God’s holy spirit was poured out upon them , the apostle Peter gave a stirring talk explaining the meaning of this miracle , and some 3,000 became beli



---


## Installation of JoeyNMT

JoeyNMT is a simple, minimalist NMT package which is useful for learning and teaching. Check out the documentation for JoeyNMT [here](https://joeynmt.readthedocs.io)  

In [17]:
# Install JoeyNMT
! git clone https://github.com/joeynmt/joeynmt.git
! cd joeynmt; pip3 install .

Cloning into 'joeynmt'...
remote: Enumerating objects: 2479, done.
remote: Total 2479 (delta 0), reused 0 (delta 0), pack-reused 2479
Receiving objects: 100% (2479/2479), 2.65 MiB | 22.40 MiB/s, done.
Resolving deltas: 100% (1732/1732), done.
Processing /content/joeynmt
     |████████████████████████████████| 51kB 1.7MB/s 
     |████████████████████████████████| 276kB 7.0MB/s 
     |████████████████████████████████| 327kB 20.1MB/s 
     |████████████████████████████████| 215kB 19.0MB/s 
     |████████████████████████████████| 102kB 9.0MB/s 
     |████████████████████████████████| 61kB 6.5MB/s 
     |████████████████████████████████| 747kB 20.5MB/s 
  Created wheel for joeynmt: filename=joeynmt-0.0.1-cp36-none-any.whl size=77296 sha256=e95e1ca891dc1cf7fa528c6f56dc2ca5df3b13abe132f3783c4daa74f0fb4b03
  Stored in directory: /tmp/pip-ephem-wheel-cache-6fb4wlz2/wheels/db/01/db/751cc9f3e7f6faec127c43644ba250a3ea7ad200594aeda70a
  Created wheel for pyyaml: filename=PyYAML-5.3.1-cp36-cp36m-lin

# Preprocessing the Data into Subword BPE Tokens

- One of the most powerful improvements for agglutinative languages (a feature of most Bantu languages) is using BPE tokenization [ (Sennrich, 2015) ](https://arxiv.org/abs/1508.07909).

- It was also shown that by optimizing the umber of BPE codes we significantly improve results for low-resourced languages [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021) [(Martinus, 2019)](https://arxiv.org/abs/1906.05685)

- Below we have the scripts for doing BPE tokenization of our data. We use 4000 tokens as recommended by [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021). You do not need to change anything. Simply running the below will be suitable. 

In [18]:
# One of the huge boosts in NMT performance was to use a different method of tokenizing. 
# Usually, NMT would tokenize by words. However, using a method called BPE gave amazing boosts to performance

# Do subword NMT
from os import path
os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language

# Learn BPEs on the training data.
os.environ["data_path"] = path.join("joeynmt", "data", source_language + target_language) # Herman! 
! subword-nmt learn-joint-bpe-and-vocab --input train.$src train.$tgt -s 4000 -o bpe.codes.4000 --write-vocabulary vocab.$src vocab.$tgt

# Apply BPE splits to the development and test data.
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < train.$src > train.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < train.$tgt > train.bpe.$tgt

! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < dev.$src > dev.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < dev.$tgt > dev.bpe.$tgt
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < test.$src > test.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < test.$tgt > test.bpe.$tgt

# Create directory, move everyone we care about to the correct location
! mkdir -p $data_path
! cp train.* $data_path
! cp test.* $data_path
! cp dev.* $data_path
! cp bpe.codes.4000 $data_path
! ls $data_path

# Also move everything we care about to a mounted location in google drive (relevant if running in colab) at gdrive_path
! cp train.* "$gdrive_path"
! cp test.* "$gdrive_path"
! cp dev.* "$gdrive_path"
! cp bpe.codes.4000 "$gdrive_path"
! ls "$gdrive_path"

# Create that vocab using build_vocab
! sudo chmod 777 joeynmt/scripts/build_vocab.py
! joeynmt/scripts/build_vocab.py joeynmt/data/$src$tgt/train.bpe.$src joeynmt/data/$src$tgt/train.bpe.$tgt --output_path joeynmt/data/$src$tgt/vocab.txt

# Some output
! echo "BPE Kikongo Sentences"
! tail -n 5 test.bpe.$tgt
! echo "Combined BPE Vocab"
! tail -n 10 joeynmt/data/$src$tgt/vocab.txt  # Herman

bpe.codes.4000	dev.en	     test.bpe.kwy    test.kwy	    train.en
dev.bpe.en	dev.kwy      test.en	     train.bpe.en   train.kwy
dev.bpe.kwy	test.bpe.en  test.en-any.en  train.bpe.kwy
bpe.codes.4000	dev.en	     test.bpe.kwy    test.kwy	    train.en
dev.bpe.en	dev.kwy      test.en	     train.bpe.en   train.kwy
dev.bpe.kwy	test.bpe.en  test.en-any.en  train.bpe.kwy
BPE Kikongo Sentences
Ng@@ ub@@ u anene ya lukwikilu ( Tala e tini kia 12 - 14 )
E mp@@ u a mv@@ it@@ a a luv@@ ul@@ uku ( Tala e tini kia 15 - 18 )
O wantu bet@@ oma yangal@@ alanga vava bemonanga e kiese tukalanga kiau kia sadila Nkand’a Nzambi ye ngolo tuv@@ anganga muna kubasadisa . ”
Ns@@ os@@ olo a mwanda ( Tala e tini kia 19 - 20 )
T@@ ut@@ omene zaya e mbeni eto , nt@@ ambu miandi ye makani mandi .
Combined BPE Vocab
Hebre@@
‛
;@@
C.@@
č@@
ā@@
Ł@@
−
Ṭ@@
û


In [20]:
# Also move everything we care about to a mounted location in google drive (relevant if running in colab) at gdrive_path
! cp train.* "$gdrive_path"
! cp test.* "$gdrive_path"
! cp dev.* "$gdrive_path"
! cp bpe.codes.4000 "$gdrive_path"
! ls "$gdrive_path"

bpe.codes.4000	dev.en	     test.bpe.kwy    test.kwy	    train.en
dev.bpe.en	dev.kwy      test.en	     train.bpe.en   train.kwy
dev.bpe.kwy	test.bpe.en  test.en-any.en  train.bpe.kwy


# Creating the JoeyNMT Config

JoeyNMT requires a yaml config. We provide a template below. We've also set a number of defaults with it, that you may play with!

- We used Transformer architecture 
- We set our dropout to reasonably high: 0.3 (recommended in  [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021))

Things worth playing with:
- The batch size (also recommended to change for low-resourced languages)
- The number of epochs (we've set it at 30 just so it runs in about an hour, for testing purposes)
- The decoder options (beam_size, alpha)
- Evaluation metrics (BLEU versus Crhf4)

In [21]:
# This creates the config file for our JoeyNMT system. It might seem overwhelming so we've provided a couple of useful parameters you'll need to update
# (You can of course play with all the parameters if you'd like!)

name = '%s%s' % (source_language, target_language)
gdrive_path = os.environ["gdrive_path"]

# Create the config
config = """
name: "{name}_transformer"

data:
    src: "{source_language}"
    trg: "{target_language}"
    train: "data/{name}/train.bpe"
    dev:   "data/{name}/dev.bpe"
    test:  "data/{name}/test.bpe"
    level: "bpe"
    lowercase: False
    max_sent_length: 100
    src_vocab: "data/{name}/vocab.txt"
    trg_vocab: "data/{name}/vocab.txt"

testing:
    beam_size: 5
    alpha: 1.0

training:
    #load_model: "{gdrive_path}/models/{name}_transformer/1.ckpt" # if uncommented, load a pre-trained model from this checkpoint
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"           # TODO: try switching from plateau to Noam scheduling
    patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 1000      # warmup steps for Noam scheduler (used with Transformer)
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0003
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 4096
    batch_type: "token"
    eval_batch_size: 3600
    eval_batch_type: "token"
    batch_multiplier: 1
    early_stopping_metric: "ppl"
    epochs: 30                     # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 1000          # TODO: Set to at least once per epoch.
    logging_freq: 100
    eval_metric: "bleu"
    model_dir: "models/{name}_transformer"
    overwrite: False               # TODO: Set to True if you want to overwrite possibly existing models. 
    shuffle: True
    use_cuda: True
    max_output_length: 100
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 3

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4             # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256   # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4              # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256    # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
""".format(name=name, gdrive_path=os.environ["gdrive_path"], source_language=source_language, target_language=target_language)
with open("joeynmt/configs/transformer_{name}.yaml".format(name=name),'w') as f:
    f.write(config)

# Train the Model

This single line of joeynmt runs the training using the config we made above

In [22]:
# Train the model
# You can press Ctrl-C to stop. And then run the next cell to save your checkpoints! 
!cd joeynmt; python3 -m joeynmt train configs/transformer_$src$tgt.yaml

2020-08-23 18:59:20,396 Hello! This is Joey-NMT.
2020-08-23 18:59:20.552448: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-08-23 18:59:22,323 Total params: 12155136
2020-08-23 18:59:22,325 Trainable parameters: ['decoder.layer_norm.bias', 'decoder.layer_norm.weight', 'decoder.layers.0.dec_layer_norm.bias', 'decoder.layers.0.dec_layer_norm.weight', 'decoder.layers.0.feed_forward.layer_norm.bias', 'decoder.layers.0.feed_forward.layer_norm.weight', 'decoder.layers.0.feed_forward.pwff_layer.0.bias', 'decoder.layers.0.feed_forward.pwff_layer.0.weight', 'decoder.layers.0.feed_forward.pwff_layer.3.bias', 'decoder.layers.0.feed_forward.pwff_layer.3.weight', 'decoder.layers.0.src_trg_att.k_layer.bias', 'decoder.layers.0.src_trg_att.k_layer.weight', 'decoder.layers.0.src_trg_att.output_layer.bias', 'decoder.layers.0.src_trg_att.output_layer.weight', 'decoder.layers.0.src_trg_att.q_layer.bias', 'decoder.layers.0.src_trg_

In [33]:
# Copy the created models from the notebook storage to google drive for persistant storage 
!cp -r joeynmt/models/${src}${tgt}_transformer/* "$gdrive_path/models/${src}${tgt}_transformer/"

In [34]:
# Copy the created models from the notebook storage to google drive for persistant storage 
!cp joeynmt/models/${src}${tgt}_transformer/best.ckpt "$gdrive_path/models/${src}${tgt}_transformer/"

In [35]:
!ls joeynmt/models/${src}${tgt}_transformer

00056000.hyps.dev   2000.hyps	33000.hyps  46000.hyps	56000.hyps
00056000.hyps.test  21000.hyps	34000.hyps  47000.hyps	6000.hyps
10000.hyps	    22000.hyps	35000.hyps  48000.hyps	7000.hyps
1000.hyps	    23000.hyps	36000.hyps  49000.hyps	8000.hyps
11000.hyps	    24000.hyps	37000.hyps  50000.hyps	9000.hyps
12000.hyps	    25000.hyps	38000.hyps  5000.hyps	best.ckpt
13000.hyps	    26000.hyps	39000.hyps  51000.hyps	config.yaml
14000.hyps	    27000.hyps	40000.hyps  52000.hyps	src_vocab.txt
15000.hyps	    28000.hyps	4000.hyps   53000.hyps	tensorboard
16000.hyps	    29000.hyps	41000.hyps  54000.ckpt	train.log
17000.hyps	    30000.hyps	42000.hyps  54000.hyps	trg_vocab.txt
18000.hyps	    3000.hyps	43000.hyps  55000.ckpt	validations.txt
19000.hyps	    31000.hyps	44000.hyps  55000.hyps
20000.hyps	    32000.hyps	45000.hyps  56000.ckpt


In [36]:
# Output our validation accuracy
! cat "$gdrive_path/models/${src}${tgt}_transformer/validations.txt"

Steps: 1000	Loss: 94990.79688	PPL: 58.15327	bleu: 1.05368	LR: 0.00030000	*
Steps: 2000	Loss: 79171.57031	PPL: 29.56052	bleu: 3.79407	LR: 0.00030000	*
Steps: 3000	Loss: 71365.70312	PPL: 21.16939	bleu: 4.73938	LR: 0.00030000	*
Steps: 4000	Loss: 65827.97656	PPL: 16.70467	bleu: 7.62276	LR: 0.00030000	*
Steps: 5000	Loss: 62261.00781	PPL: 14.34092	bleu: 9.52336	LR: 0.00030000	*
Steps: 6000	Loss: 59579.26172	PPL: 12.78675	bleu: 10.86494	LR: 0.00030000	*
Steps: 7000	Loss: 57351.00781	PPL: 11.62432	bleu: 12.41515	LR: 0.00030000	*
Steps: 8000	Loss: 55338.14453	PPL: 10.66537	bleu: 13.34303	LR: 0.00030000	*
Steps: 9000	Loss: 53618.95312	PPL: 9.90923	bleu: 14.68891	LR: 0.00030000	*
Steps: 10000	Loss: 52201.00391	PPL: 9.32609	bleu: 15.15425	LR: 0.00030000	*
Steps: 11000	Loss: 50936.39453	PPL: 8.83503	bleu: 15.95811	LR: 0.00030000	*
Steps: 12000	Loss: 50030.32031	PPL: 8.49917	bleu: 17.10952	LR: 0.00030000	*
Steps: 13000	Loss: 49374.60938	PPL: 8.26410	bleu: 17.06075	LR: 0.00030000	*
Steps: 14000	Loss:

In [37]:
# Test our model
! cd joeynmt; python3 -m joeynmt test "$gdrive_path/models/${src}${tgt}_transformer/config.yaml"

2020-08-23 21:35:15,307 Hello! This is Joey-NMT.
/content/joeynmt/joeynmt/search.py:351: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:766.)
  finished_hyp = is_finished[i].nonzero().view(-1)
2020-08-23 21:35:42,245  dev bleu:  26.69 [Beam search decoding with beam size = 5 and alpha = 1.0]
2020-08-23 21:36:22,367 test bleu:  36.28 [Beam search decoding with beam size = 5 and alpha = 1.0]
